# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [34]:
# import libraries

import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sqlalchemy import create_engine

from sklearn.multioutput import MultiOutputClassifier

import pandas as pd


from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
#engine = create_engine('sqlite:///InsertDatabaseName.db')


In [36]:
#df=pd.read_sql_table("InsertTableName",con=engine)

In [37]:
#df

In [62]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df=pd.read_sql_table("InsertTableName",con=engine)
df=df.drop(columns=['original'], axis=1)
df.dropna(axis=0,inplace=True)
#X = df[['id','message','genre']]
X = df[['message']]
Y = df.drop(columns=['id','message','genre'],axis=1)

In [63]:
#X['message'].values[5]

In [64]:
#df=df.drop(columns=['original'], axis=1)

In [65]:
#df.dropna(axis=0,inplace=True)

In [66]:
#df.isnull().sum()

In [67]:
#X.id.unique()

In [68]:
#Y.head()

In [69]:
#Y.isnull().sum()

In [70]:
Y.shape

(26207, 36)

In [71]:
#Y.dropna(axis=0,inplace=True)

In [72]:
X.shape,Y.shape

((26207, 1), (26207, 36))

### 2. Write a tokenization function to process your text data

In [73]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    tokens=[word.lower() for word in tokens if word.isalpha()]

    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [74]:
#tokenize(X)

In [75]:
#for messages in X['message']:
#    print(tokenize(messages))

In [76]:
X['message'].isnull().sum()

0

In [77]:
tokenize(X['message'][3])

['un',
 'report',
 'leogane',
 'destroyed',
 'only',
 'hospital',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [78]:
'''
pipeline = Pipeline([
#    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        #])
        #,

#        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
'''
pipeline = Pipeline([
#    ('features', FeatureUnion([

        
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
        #])
        #,

#        ('starting_verb', StartingVerbExtractor())
    

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])




In [79]:
parameters = {
'vect__ngram_range': ((1, 1), (1, 2)),
#'text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
'tfidf__use_idf': (True, False),
#'clf__n_estimators': [50, 100, 200],
#'clf__min_samples_split': [2, 3, 4],
}


In [80]:
cv = GridSearchCV(pipeline, param_grid=parameters)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [81]:
#Y[Y.isnull()]

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [90]:
X_train.shape,y_train.shape

((19655, 1), (19655, 36))

In [91]:
X_train.head()

,message
11751,Santiago airport terminal to reopen in 12-15 d...
25109,"The U.S. rescue teams, comprised of approximat..."
6712,"I need a tent, medicine, rice, potable water. ..."
23556,"The Red Cross Team on the ground, including Ne..."
7603,Is there a always bus please answer?


In [93]:
pipeline.fit(X_train,y_train)
#pipeline.fit(X,Y)



ValueError: Found input variables with inconsistent numbers of samples: [1, 19655]

In [94]:
y_train.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.